# SNAM

## Data reading

In [1]:
import pandas as pd
df = pd.read_excel('dataset/Percentuali_di_prelievo_AT_2022-2023.xls')
df = df.rename(columns={"Unnamed: 1": "day_of_week", "Data": "date"})
df['day_of_week'] = pd.to_datetime(df['day_of_week']).dt.day_name()

In [4]:
df.columns

Index(['date', 'day_of_week', 'c1%B1', 'c1%C1', 'c1%D1', 'c1%E1', 'c1%F1',
       'c1%B2', 'c1%C2', 'c1%D2', 'c1%E2', 'c1%F2', 'c1%B3', 'c1%C3', 'c1%D3',
       'c1%E3', 'c1%F3', 'c2%', 'c4%', 't1%1', 't1%2', 't1%3'],
      dtype='object')

C1 Riscaldamento  
C2 Uso cottura cibi e/o produzione di acqua calda  
C3 Riscaldamento + uso cottura cibi e/o produzione acqua calda sanitaria  
C4 Uso condizionamento  
C5 Uso condizionamento + riscaldamento   

T1 Uso tecnologico (artigianale-industriale)  
T2 Uso tecnologico + riscaldamento  

In [5]:
mapping_name_to_type = {
    "c1": "riscaldamento",
    "c2": "cottura+acqua_calda",
    "c3": "riscaldamento + cottura + acqua calda",
    "c4": "condizionamento",
    "c5": "condizionamento + riscaldamento",
    "t1": "uso tecnologico (artigianale-industriale)",
    "t2": "uso tecnologico + riscaldamento"
}

In [6]:
df.columns.str[:2].unique()

Index(['da', 'c1', 'c2', 'c4', 't1'], dtype='object')

Interpretazione del dataset:
- ogni riga è una giornata relativa al consumo di gas
- ogni colonna è una persona/cliente diverso
- il nome della colonna indica il tipo di consumo di quel cliente secondo il mappaggio sopra definito

In [23]:
(
    df.columns[df.columns.str.contains('c1')].shape, 
    df.columns[df.columns.str.contains('c2')].shape, 
    df.columns[df.columns.str.contains('c4')].shape, 
    df.columns[df.columns.str.contains('t1')].shape,
    df.columns[df.columns.str.contains('day|date')].shape,
    df.columns.shape
)

((15,), (1,), (1,), (3,), (2,), (22,))

Abbiamo a disposizione:
- 15 clienti a consumo C1 = solo riscaldamento
- 1 cliente a consumo C2 = cottura + acqua calda
- 1 cliente a consumo C4 = condizionamento
- 3 clienti a consumo T1 = uso tecnologico (artigianale-industriale)"

Non è chiaro se questi siano obbligatoriamente persone diverse, o alcuni di questi profili di consumo appartengono allo stesso.  
Si potrebbe provare a controllare se i profili C2, C4, T1 facciano riferimento ai profili C1 di cui disponiamo.  
Per effettuarlo potremmo controllare come si muovono insieme queste serie rispetto a quelle di C1 con una cross-correlazione.

In [13]:
df.drop(columns=['date', 'day_of_week']).sum(axis=0)

c1%B1    100.0
c1%C1    100.0
c1%D1    100.0
c1%E1    100.0
c1%F1    100.0
c1%B2    100.0
c1%C2    100.0
c1%D2    100.0
c1%E2    100.0
c1%F2    100.0
c1%B3    100.0
c1%C3    100.0
c1%D3    100.0
c1%E3    100.0
c1%F3    100.0
c2%      100.0
c4%      100.0
t1%1     100.0
t1%2     100.0
t1%3     100.0
dtype: float64

La il valore j della i-esima colonna, è la percentuale del totale della colonna. Se leggo 0,00010. E' lo 0,00010%.

In [15]:
df['c1%B1'].max()

np.float64(1.0583294467900397)

# Obiettivi analisi descrittiva
0. Quali sono le caratteristiche principali di questo dataset?
1. Quali sono le principali differenze e similitudini fra i diversi profili in termini di stagionalità?
2. Partendo dall’analisi del punto b, a quale tipologia di utilizzo potrebbe realisticamente essere
attribuito ogni profilo?
3. Ci possono essere a tuo parere delle variabili che possono ragionevolmente impattare il
profilo di consumo di un cliente gas che in questi profili non verrebbero considerate?

# Domanda 0
0. Quali sono le caratteristiche principali di questo dataset?

In [11]:
df.describe()

,date,c1%B1,c1%C1,c1%D1,c1%E1,c1%F1,c1%B2,c1%C2,c1%D2,c1%E2,...,c1%B3,c1%C3,c1%D3,c1%E3,c1%F3,c2%,c4%,t1%1,t1%2,t1%3
count,365,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,...,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,3.650000e+02,365.000000,3.650000e+02,365.000000,3.650000e+02,3.650000e+02
mean,2023-04-01 00:00:00,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,...,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,2.739726e-01,0.273973,2.739726e-01,0.273973,2.739726e-01,2.739726e-01
min,2022-10-01 00:00:00,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.019865e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.158557,1.000000e-08,0.171303,1.000000e-08,1.000000e-08
25%,2022-12-31 00:00:00,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,2.453621e-02,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,0.188991,1.000000e-08,0.272717,2.713328e-01,1.000000e-08
50%,2023-04-01 00:00:00,1.000000e-08,1.000000e-08,1.000000e-08,1.545755e-01,1.573372e-01,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,...,1.000000e-08,1.000000e-08,1.000000e-08,1.000000e-08,3.583823e-02,0.237408,1.000000e-08,0.279590,3.263408e-01,3.896684e-01
75%,2023-07-01 00:00:00,6.511353e-01,5.741217e-01,5.598017e-01,5.469917e-01,5.063921e-01,6.245658e-01,6.085329e-01,5.858121e-01,5.605555e-01,...,1.000000e-08,5.911089e-01,5.803681e-01,5.462693e-01,5.317976e-01,0.366369,7.211932e-01,0.289368,3.450471e-01,4.167021e-01
max,2023-09-30 00:00:00,1.058329e+00,9.146862e-01,8.867823e-01,8.247798e-01,7.716074e-01,1.273638e+00,1.068104e+00,1.061207e+00,9.902603e-01,...,1.537172e+00,1.289829e+00,1.282510e+00,1.198137e+00,1.129772e+00,0.440201,9.757592e-01,0.316521,3.860725e-01,4.742298e-01
std,NaN,4.038501e-01,3.459846e-01,3.191266e-01,3.030962e-01,2.700257e-01,4.590206e-01,3.877195e-01,3.666473e-01,3.522351e-01,...,5.186676e-01,4.439024e-01,4.224604e-01,4.068848e-01,3.759001e-01,0.097232,3.962786e-01,0.029027,1.257020e-01,1.884016e-01


Il dataset in analisi si presenta come un dataset dove ogni riga rappresenta una data, ed ogni colonna rappresenta un profilo di consumo di un cliente.  
All'i-esima riga della j-esima colonna abbiamo il valore di percentuale di consumo di gas all'interno dell'anno termico di considerazione. Quindi il valore in considerazione e la percentuale rispetto al totale della sua colonna.  
Infatti sommando le colonne otteniamo valori pari al 100%.  
A seguito di [ricerche](https://www.basengassrl.it/scambio-di-info-tra-gli-operatori/tisg-obblighi-informativi-di-cui-all-art-7-8/) fatte online risulta che le sigle c1, c2, c4, t1 facciano riferimento già a diversi profili di consumo. Possiamo assumere che c1%C1 e c1%D1 siano il consumo di tipo C1 di due clienti diversi.  
E' chiaro che le colonne indicate come c2 e c4 facciano riferimento a tipologie diverse di profili di consumo, non è chiaro se diverse tipologie di profili possano far riferimento allo stesso cliente. Per esempio non è chiaro se il c2 che vediamo, essendo uno solo, possa essere il consumo c2 del profilo c1%D1.  
Dai seguenti grafici andremo a vedere le distribuzioni tramite una serie di box plot per osservare eventuali somiglianze dal punto di vista dei valori, tenendo sempre a mente che stiamo lavorando con valori percentuali.

## Analisi delle distribuzioni
### Tutti i profili insieme

In [38]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
c1_columns = df.columns.drop(['date','day_of_week'])
for c1_col in c1_columns:
    fig.add_trace(
        go.Box(
            x=df[c1_col],
            name=c1_col
        )
    )
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)
fig.show()

Tramite questo primo grafico di confronto possiamo osservare come i vari profili hanno delle diverse distribuzioni e quartili, nonostante i loro valori vadano tutti dallo 0 al 100% a livello teorico essendo delle perncetuali.  
Possiamo osservare come tutti abbiano come valori minimo 10^-8, tuttavia il valore massimo ovvero la percentuale di gas rispetto al loro consumo all'interno dell'anno termico vari particolarmente, sia all'interno della stessa tipologia di profili, sia tra profili diversi.  
Per esempio possiamo osserare come il valore massimo del profilo **c1%C3** sia pari a 1.28%, indicando come all'interno di un singolo giorno questo profilo ha consumato poco più dell'1% del suo anno termico. Possiamo osservare come però la sua mediana, il valore al primo quartile ed il valore minimo siano tutti pari 10^-8; indicando come il consumo di questo particolare profilo sia particolarmente irregolare.  

In generale possiamo osservare come le distribuzioni di questi profili siano asimmetriche. Possiamo fare due plot di esempio come segue tenendo come campionii profli c1%C3 e c1%F3:

In [50]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

c1_columns = ['c1%C3', 'c1%F3']
fig = make_subplots(rows=2, cols=1,  subplot_titles=("Distribution " + c1_columns[0], "Distribution "+ c1_columns[1]))

for i, c1_col in enumerate(c1_columns):
    fig.add_trace(
        go.Histogram(
            x=df[c1_col], 
            name=c1_col, 
        ),
        col=1,
        row=i+1,  
    )
    
    
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)

fig.show()

Osserviamo come le loro distribuzioni siano diverse tra di loro, ma che entrambe presentino una cera assimmetria.

### C1

In [30]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
c1_columns = df.columns[df.columns.str.contains('c1')]
for c1_col in c1_columns:
    fig.add_trace(
        go.Box(
            x=df[c1_col],
            name=c1_col
        )
    )

fig.show()

### C2 e C4 

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
c1_columns = df.columns[df.columns.str.contains('c2|c4')]
for c1_col in c1_columns:
    fig.add_trace(
        go.Box(
            x=df[c1_col],
            name=c1_col
        )
    )

fig.show()

Nel dettaglio quindi possiamo aggiungere come considerazione che il profilo C2 sia meno assimmetrico rispetto al C4  
Tuttavia che il c4 sia assimmetrico ce lo possiamo aspettare poiché secondo le ricerche risulta essere il profilo di consumo relativo al condizionamento, che è attivo solo per i mesi estivi.

### T1

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
c1_columns = df.columns[df.columns.str.contains('t1')]
for c1_col in c1_columns:
    fig.add_trace(
        go.Box(
            x=df[c1_col],
            name=c1_col
        )
    )

fig.show()

All'interno del t1, la categoria industriale ed artigianato possiamo osservare come la distribuzione sia particolarmente diversa tra i vari profili. Nel dettaglio entreremo andando ad osservare i grafici delle serie storiche.

## Correlazioni

In [53]:
corr_matrix = df.drop(columns=['date','day_of_week']).corr()


In [78]:
import plotly.graph_objects as go

fig = go.Figure(
    data=go.Heatmap(
        z=corr_matrix,
        x=corr_matrix.columns.tolist(),
        y=corr_matrix.index.tolist(),
        text=corr_matrix.round(2),
        hoverongaps = False,
        texttemplate="%{text}",
        textfont={"size":10}
    )
)

fig.update_layout(
    title='Correlation between profiles',
    autosize=False,
    width=1200,
    height=1200,
)

fig.show()

Dalla matrice di correlazione tra le time series possiamo andare ad osservare come esistano gruppi di profili altamente correlati tra di loro, ed altri che differiscono particolarmente.  
Un esempio è il gruppo di profili c1%C1, c1%D1, c1%E1, c1%F1. Questo ci permette di ottenere idea di come i profili in esame presentino caratteristiche simili tra di loro a gruppi.  
Nel dettaglio osserveremo direttamente questa situazione controllando i profili delle serie temporali 1 ad 1. 

# Domanda 1 e 2
1. Quali sono le principali differenze e similitudini fra i diversi profili in termini di stagionalità?
2. Partendo dall’analisi del punto b, a quale tipologia di utilizzo potrebbe realisticamente essere
attribuito ogni profilo?

## C1

In [27]:
import plotly.graph_objects as go
# Create traces
fig = go.Figure()

c1_columns = df.columns[df.columns.str.contains('c1')]

for c1_col in c1_columns:
    fig.add_trace(
        go.Scatter(
            x=df['date'], 
            y=df[c1_col],
            mode='lines',
            name=c1_col,
            text=df['day_of_week']
        )
    )

fig.show()

I c1 si comportano come ci si aspetterebbe da dei profili di riscaldamento: partono bassi all'inizio della stagione autunnale per poi crescere gradualmente con un picco a gennaio, per poi tornare a scendere verso giugno e luglio. Un esempio è il profilo **c1F3** segue questo andamento. Per questo profilo, si noti che il valore del riscaldamento ha degli azzeramenti nel corso di quello che è il weekend o durante quelli che possono essere giorni di chiusura di un ufficio, come l'8 gennaio che è festività nazionale in Italia.
  
Ci sono poi altri profili che hanno comportamenti analoghi, ma all'interno di range temporali più stretti. Come il **c1C3** che inizia ad avere un valore rilevante a Novembre, crescere fino a gennaio e poi scendere fino ad azzerarsi a maggio. Sì noti che il valore del riscaldamento ha degli azzeramenti nel corso di quello che è il weekend anche per questo profilo. Possiamo quindi dedurre che c1F3 e c1C3 abbiano dei comportamenti simili nel weekend ma differiscano per i mesi di stagionalità-

Abbiamo invece **c1F1** che si comporta come descritto per gli altri nel range ottobre, luglio. La differenza rispetto agli altri 2 profili d'esempio è che non ha azzeramenti del weekend. Facendo dedurre che l'uso del riscaldamento sia diverso in questi profili e come possa essere diversa la ragione di utilizzo e la tipologia di edificio riscaldato.

A questo punto possiamo fare delle ipotesi.
Si potrebbe ipotizzare che quei profili per cui il riscaldamento si azzera nel weekend si tratti di profili relativi ad **uffici**, magazzini o comunque qualsiasi tipologia di struttura che nel weekend non ha necessità di rimanere riscaldata quando non sono presenti i lavoratori.  
Mentre quei profili per cui l'azzeramento del consumo nel weekend siano relativi a **case** o abitazioni di uso comune.  
Il fatto che non tutti abbiano la fase iniziale crescita, picco, e decrescita nelle stesse date può essere dovuto al fatto che il riscaldamento non è uguale in tutte le zone d'italia, e che le leggi di attivazione del riscaldamento siano di comuni e/o regioni diverse. Per esempio a Milano: "periodo di accensione: dal 22 ottobre 2023 all'8 aprile 2024" [X](https://www.comune.milano.it/aree-tematiche/ambiente/energia/calendario-accensione-impianti)

Di fronte a queste breve considerazioni abbiamo la possibilità di assumere che:
- i profili osservati appartengano a case (o edifici usati come dimora) quando nel weekend non si hanno azzeramenti
- appartengono ad altre strutture inattive nel weekend (che può essere solo la domenica, o sia sabato che domenica) quando si hanno azzeramenti (magazzini, posti di lavoro)
- i profili osservati appartengono a zone geografiche diverse di italia e non provengono tutti dalla medesima.

NB: Notostante viene usata la parola azzerarsi per definire un crollo del consumo di gas, il valore non scende mai sotto lo 10e-8%.

## C2 E C4

In [28]:
import plotly.graph_objects as go
# Create traces
fig = go.Figure()

c1_columns = df.columns[df.columns.str.contains('c2|c4')]

for c1_col in c1_columns:
    fig.add_trace(
        go.Scatter(
            x=df['date'], 
            y=df[c1_col],
            mode='lines',
            name=c1_col,
            text=df['day_of_week']
        )
    )

fig.show()

Dalle ricerche online abbiamo ricavato che c2 è l'indicatore relativo al consumo di acqua calda e gas per cottura.  
Possiamo osservare come il profilo della serie temporale infatti non si azzeri mai. Inoltre questo cresce nel periodo Ottobre-Gennaio per poi descrescere nel periodo successivo. Questo può essere ipotizzato come la parte di consumo relativa al consumo dell'acqua calda, utilizzata maggiormente nella fine del periodo Autunnale e per tutta la parte invernale, per poi descrescere gradualmente in primavera.  
Qusto **c2** a causa dei presunti aumenti dovuti all'utilizzo dell'acqua calda, suggerisce come si tratti del consumo di gas di un'abitazione, piuttosto che di un ufficio o altro; anche poiché non presenta il comportamento da ufficio che abbiamo osservato all'interno di c1.

Il **c4** dalle ricerche risulta essere il profilo del consumo relativo ai condizionatori. Infatti questo è al minimo per tutta la parte invernale fino al 31 maggio, per poi essere attivato costantemente con qualche decrescita giornaliera, forse nei giorni meno caldi od in cui si è stati meno in casa.

## T1

In [138]:
import plotly.graph_objects as go
# Create traces
fig = go.Figure()

c1_columns = df.columns[df.columns.str.contains('t1')]

for c1_col in c1_columns:
    fig.add_trace(
        go.Scatter(
            x=df['date'], 
            y=df[c1_col],
            mode='lines',
            name=c1_col,
            text=df['day_of_week']
        )
    )

fig.show()

I profili hanno le seguenti caratteristiche:
- **t1_1**: profilo mai vicino allo 0 e generalmente stabile nel corso del tempo. Ha delle riduzioni forti nelle prossimità delle festività. Per esempio ha delle riduzioni dal 31/10 al 6/11, il ponte dei morti. Ha una prima riduzione dal 19/12 al 25/12 per poi diminuire ulteriormente dal 26/12 al 1/01 (festività di natale e fine anno) per poi gradualmente ricrescere e tornare stabile. Non ha dei picchi negativi nei weekend. Il profilo t1 si riferisce ad uso tecnologico (artigianale + industriale). Dovrebbe trattarsi quindi per sua definizione di un profilo industriale di un edificio che nel weekend non chiude, dove l'edificio resta attivo e consuma gas per motivi industriali. Per esempio potrebbe trattarsi di una qualche azienda di produzione industriale dove la catena di produzione non si fermai mai ed impiega dei macchinari a gas.
- **t1_2**: diversamente dal profilo t1_1, presenta degli azzeramenti del consumo nel weekend. Quindi dovrebbe trattarsi di un uso industriale dove l'edificio chiude la domenica (e non il sabato). Potrebbe trattarsi di un impianto artigianale o di un azienda dove ci sono i turni ed i lavoratori hanno solo la domenica come giorno libero. Presenta dei consumi più costanti rispetto al t1_1 durante la maggior parte dell'anno.
- **t1_3**: si presenta come un profilo simile al profilo di t1_2. Con la differenza che l'azienda chiude anche il sabato.

# Domanda 3
3. Ci possono essere a tuo parere delle variabili che possono ragionevolmente impattare il
profilo di consumo di un cliente gas che in questi profili non verrebbero considerate?

Osservando il comportamento di una serie temporale all'interno di quali periodi si attiva, se è attiva nel weekend e di quale tipologia di profilo si tratta siamo riusciti a dividere in diverse categorie di serie temporali.
Abbiamo ottenuto per i profili C1:
- riscaldamenti attivi tutta la settimana
- riscaldamenti non attivi per il weekend
- riscaldamenti che sono attivi in momenti diversi dell'anno e presentano differenza nei mesi di attivazione
Abbiamo ottenuto per i profili T1:
- consumi attivi tutta la settimana
- consumi non attivi per tutto il weekend
- consumi non attivi solo la domenica

Per quanto riguarda i profili C1 sarebbe utile conoscere la posizione geografica, con granularità regionale, provinciale o ancor meglio comunale del profilo di riscaldamento, in modo da poter raggruppare i comportamenti che sono vincolati per legge a seguire determinati vincoli di accensione e non doversi affidare esclusivamente alla stagionalità osservata nel passato per ogni serie temporale.  
Unito alla posizione geografica, potrebbe essere utile avere informazioni metereologiche relative alla singola zona geografica in modo da poter regolare di conseguenza la previsione del consumo del profilo; in modo da catturare giorni invernali particolarmente caldi o giorni primaverili/autunnali particolarmente freddi per cui il consumo verrà presumibilmente regolato di conseguenza.

Per quanto riguarda i profili T1 potremmo aver la necessità di distiguere attività industriali/artigianali che appartengono a settori diversi, in modo da poter meglio raggruppare comportamenti simili di profili di consumo. Per esempio un industria che utilizza macchinari pesanti a base di gas come forni industriali ogni giorno avrà sicuramente necessità diverse rispetto un'attività industriale che non usa macchine di questo tipo con costanza o non le usa affatto, presentando sicuramente degli andamenti diversi nel consumo percentuale. Ci si aspetta la prima che abbia un consumo costante ed una percentuale di profilo egualmente distribuita; mentre il secondo potrebbe presentare dei picchi nel giorno in cui utilizza tale macchinario. Identificare quindi la tipologia di industria di riferimento, insieme a quali macchinari e tipologia disponga potrebbe permettere di identificare e prevedere meglio determinati comportamenti.  

In aggiunta a ciò, determinare quali sono le festività nazionali permette di identificare quei giorni che definiremo "di lavoro" ma che presenteranno delle riduzioni rispetto ad un ipotetico valore atteso per i profili di consumo di tipo T1 e di tipo C1 che sono relativi non a case abitate ma ad uffici o fabbriche.

Per quanto riguarda i profili C2 potrebbe essere importante identificare quali facciano riferimento ad abitazioni, od a strutture diverse dove sia più frequente l'uso di acqua calda poiché parte di un servizio destinato a clienti, come palestre fornite di docce, o ancor di più nel caso di impianti termali. Per questi in particolare ci si aspetta un profilo di consumo diverso da quanto osservato nel grafico di esempio, ma un consumo di acqua calda più costante.

# Modellazione
Proporre un metodo/algoritmo/modello per proiettare i profili forniti da Snam sul futuro.  
Lo scopo è arrivare ad avere per ogni tipologia di utilizzo un profilo per ogni giorno del 2024, che possa essere moltiplicato per il consumo annuo (dato esternamente) e fornire così una prima proxy del consumo
giornaliero di quel cliente per il prossimo anno.  
La scelta della tipologia di algoritmo (applicazioni di
regole statiche, modello classico statistico o modelli di ML) è a libera valutazione del candidato.  
L’implementazione pratica di questo punto in Python è assolutamente opzionale, la cosa importante
è scrivere nel notebook un’accurata descrizione della soluzione proposta, dei passaggi logici che
seguiresti e delle possibili strategie/metriche per valutarne l’efficacia.  

## Primo modello - SARIMA e SARIMAX

Dati i dati in ingresso, abbiamo diverse serie temporali, 1 per ogni profilo di consumo, di durata pari ad un anno termico.  
L'obiettivo è avere 1 singolo modello in grado di stimare il comportamento di una serie temporale.  
I profili di consumo che abbiamo osservato nell'analisi esplorativa sono profili che presentano tutti una stagionalità, ma la stagionalità non è sempre la stessa. Il vantaggio che abbiamo è che questi non presentano però un trend di crescita da dover stimare separatamente.
Un primo approccio che possiamo seguire per poter tentare di prevedere un valore temporalmente successivo è l'utilizzo di un modello in grado di tenere conto della stagionalità, quindi l'approccio classico delle serie temporali è l'utilizzo dei modelli SARIMA e SARIMAX. La differenza tra i due è che il SARIMA utilizza solo il valore della serie temporale, mentre il SARIMAX utilizza anche feature che non sono i valori alla serie temporale.


![SARIMAX EQUATION](asset/sarima_equation.png)


Scelto questo tipo di modello, dobbiamo tenere conto di due problematiche:
- il SARIMAX da solo lavora su un'unica serie temporale con le sue caratteristiche, mentre dalla nostra analisi esplorativa abbiamo osservato come siano diversi e variegati i comportamenti dei profili di consumo
- noi cerchiamo di prevedere valori che sono una percentuale tra 0 e 100
- col SARIMAX prevediamo un valore alla volta 

**Numero di modelli**  
Per risolvere il primo problema possiamo scegliere due strategie:
- sviluppare 1 solo modello generico ed osservare come si comporta, introducendo delle feature X che differenzino la tipologia di serie temporale con cui il modello sta ragionando
- sviluppare N modelli di previsione, uno per ogni tipologia di profilo di consumo che identifichiamo

Possiamo seguire entrambe le strade e valutare quale risulti essere la più efficiente.

**Feature preprocessing**  
Per quanto riguarda il secondo problema, ed evitare di incorrere in problematiche di ottenere previsioni al di fuori dell'intervallo di valori possibili, possiamo occuparci di fare un preprocessing iniziale dove rimappiamo il valore delle numeriche tra 0 e 100 tra -inf, +inf. Successivamente una volta ottenuta la previsione possiamo ritrasformare correttamente all'interno dell'intervallo di valori possibili per ottenere il valore percentuale di nostro interesse.
Inoltre il nostro modello deve garantire che la somma di tutte le previsioni sia pari a 100% e non ecceda questo valore.

**Processo iterativo**  
Col SARIMAX siamo in grado di base di prevedere un valore alla volta che può essere un valore successivo di 1 passo, 2 passi, o N passi. Il nostro obiettivo però è riuscire a prevedere completamente i valori dell'anno successivo.
Quindi andremo a prevedere ben da 1 fino a 365 valori futuri.
A questo punto anche qui le strategie possibili sono:
- sviluppo un singolo modello in grado di prevedere il passo 1, e per prevedere il valore al passo 2 utilizzo il medesimo modello insieme alla previsione ottenuta al passo 1 come se fosse una feature veritiera
- sviluppo 365 modelli uno indipendente l'uno dall'altro per prevedere il giorni in modo separato.
- un trade off tra le due scelte, come sviluppare 7 modelli che predicono ognuno il giorno della settimana in modo indipendente tra di loro e poi prevedere la settimana successiva usando questi 7 valori predetti.
Lo svantaggio del primo è che il modello diventa dipendente su sè stesso, lo svantaggio del secondo è che il numero di modelli da creare tende ad esplodere di numero. Il trade off tra i due potrebbe essere un approccio valido.

**Semplificazione del problema**    
Possiamo semplificare il problema? E' davvero necessario andare a predirre ogni giorno della settimana in modo separato? Andiamo ad osservare la seguente situazione.

In [137]:
import plotly.graph_objects as go
df_with_full_weeks = df.loc[2:358]
df['week_number'] = df['date'].apply(lambda x: x.isocalendar().week)
profiles_columns = df.columns[df.columns.str.contains('c1|c2|c4|t1')]
agg_dict = {profile:pd.Series.nunique  for profile in profiles_columns}
grouped = df.groupby("week_number").agg(agg_dict).reset_index()
melted = pd.melt(grouped, id_vars='week_number', value_vars=profiles_columns)

average_number_of_values_per_week_per_profile = melted.groupby(['variable'])['value'].mean().reset_index()

fig = go.Figure(
    go.Bar(
        y=average_number_of_values_per_week_per_profile['value'],
        x=average_number_of_values_per_week_per_profile['variable'],
        orientation='v'
    )
)

fig.update_layout(
    title='Media di valori distinti per settimana per profili'
    
)

fig.show()

Tramite questo barplot possiamo osservare come in media, all'interno di una settimana i valori del consumo percentuale siano tra l'1 ed il 2. Questo ci permette di capire che all'interno di una settimana, il consumo è costante, ad eccezione di quei profili che per un giorno o per tutto il weekend diminuisce.  
Questa osservazione ci permette di semplificare il problema, perché al posto di predirre un valore giornaliero, possiamo utilizzare un modello che predica il valore di una settimana, tenendo a mente che alcuni consumi si azzerano completamente nel weekend.  
Una volta ottenuta la predizione della settimana dobbiamo solo avere modo di riscalare questo valore per il weekend e per le festività.  
A questo punto possiamo formulare il problema nel seguente modo.


### Preparazione dei dati
Data il valore di consumo C al tempo t, definiamo come C_ww(t) il valore assunto nei 5 giorni della settimana lavorativa della settimana di cui il giorno t fa parte (è una costante per 5 giorni).
Dato il valore di consumo C al tempo t, definiamo come C_sat(t) il valore assunto il sabato di quella settimana.
Dato il valore di consumo C al tempo t, definiamo come C_sun(t) il valore assunto la domenica di quella settimana.

Trasformare il dataset in modo da ottenere seguenti feature e target per ogni riga:
- variabile target C_ww(t): il consumo da prevedere all'interno della settimana lavorativa in cui il giorno t fa parte
- variabile target C_sat(t): il consumo da prevedere del sabato della settimana di cui fa parte il giorno t 
- variabile target C_dom(t): il consumo da prevedere della domenica  della settimana di cui fa parte il giorno t
- valore del consumo ai tempi C_ww(t-1) (il consumo della settimana lavorativa precedente)
- valore del consumo ai tempi C_sat(t-1) (i consumi del sabato precedente)
- valore del consumo ai tempi C_sun(t-1) (i consumi della domenica precedente)
- valore del consumo ai tempi C_ww(t-365) (i consumi della settimana corrente l'anno passato)
- valore del consumo ai tempi C_sat(t-365) (i consumi del sabato dell'anno passato)
- valore del consumo ai tempi C_sun(t-365) (i consumi della domenica dell'anno passato)
<!-- - flag is_weekday: valorizzato ad 1 se il giorno t è un giorno della settimana lavorativa, 0 altrimenti
- flag is_saturday: valorizzato ad 1 se il giorno t è un sabato, 0 altrimenti
- flag is_weekday: valorizzato ad 1 se il giorno t è una domenica, 0 altrimenti
- flag is_holiday: valorizzato ad 1 se il giorno t è un -->

Si noti che all'interno del dataset per semplificità le varie variabili sono state indicate con t, t-1, e t-365. Ma nell'atto pratico andremo a fare fine tuning scegliendo se usare anche variabili con passi t-2, t-3,... e t-(365+7), t-(365+14), ... e così via.  
Inoltre se avessimo a disposizione altre variabili, come quelle descritte per la domanda 3 le potremmo utilizzare nel training.

### Modelli da sviluppare:
A questo punto possiamo sviluppare 3 modelli:
1. Il primo modello si occupa di prevedere C_ww(t)
2. Il secondo modello si occupa di prevedere C_sat(t)
3. Il terzo modello si occupa di prevedere C_dom(t)

Questo ci permette di ridurre il numero di modelli utilizzati a 3.  
Per addestrare questi modelli possiamo utilizzare come funzione di valutazione l'MSE e l'RMSE durante la fase di training.  
Una volta ottenuta la previsione questa deve essere rimappata nel range [0, 100] e valutata nuovamente tramite l'MSE.  
Inoltre deve essere anche valutato il fatto che la somma delle previsioni sull'anno faccia 100.

A questo punto del problema resta solo da valutare i giorni di vacanza come possono essere modificati poiché il modello non ne tiene in considerazione, siccome lavora a granualità settimanale e non più giornaliera.
Il comportamento dei vari profili all'interno del giorno feriale varia in base al fatto che:
- si tratti di riscaldamento
- si tratti di acqua calda
- si tratti di condizionamento
- si tratti di un uso industriale.
Per il riscaldamento dipende esclusivamente se si tratta di uso casalingo o per lavori:
- nel caso casalingo il consumo dovrebbe restare invariato
- nel caso lavorativo questo crolla come se fosse un giorno di non lavoro


## Secondo modello - GLM [0, 100]